In [100]:
# # reading harvardx data from the csv
# # still a work in progress since their dataset is malformatted
# import csv

# # load data
# csvfile_path = '../datasets/corpus_HarvardX_LatestCourses_based_on_2016-10-18.csv'

# with open(csvfile_path, 'r') as csvfile:
#     reader = csv.DictReader(csvfile)
#     print reader.next()

In [1]:
# search (once index already generated)
from whoosh.index import open_dir
from whoosh.query import *
from whoosh.qparser import MultifieldParser

# reopen index
index_path = 'whoosh_indices'
index = open_dir(index_path)

# query parser so we can enter normal english language queries
fields = ["title", "subtitle", "expected_learning", "syllabus", "summary", "short_summary"]
parser = MultifieldParser(fields, index.schema)

with index.searcher() as searcher:
    # run a test search
    myquery = parser.parse("neural networks")
    results = searcher.search(myquery)
    print list(results)
    
    

[<Hit {'slug': u'deep-learning--ud730', 'title': u'Deep Learning'}>, <Hit {'slug': u'machine-learning--ud262', 'title': u'Machine Learning'}>]
